In [1]:
%load_ext autotime

In [2]:
#import modules
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\ProgramData\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


time: 1.88 s


In [3]:
# prepare NLTK Stop words
from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(nltk_words)
stop_words.extend(['aquarium','atlanta','georgia'])

time: 310 ms


In [4]:
#import data
df=pd.read_excel('20180821_Tatvam_Export.xlsx')
#Remove NaN from Comments
df=df.dropna(subset=['Review Comment'])
#Remove Duplicates
df=df.drop_duplicates(subset=['Unique ID'])

time: 2.31 s


In [5]:
# Convert to list
data = df['Review Comment'].values.tolist()

time: 999 µs


In [6]:
#tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

time: 709 ms


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

C:\ProgramData\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


time: 3.44 s


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ',  'ADV','VERB']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

time: 19 ms


In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load('en')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ',  'ADV','VERB'])


time: 1min 36s


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

time: 998 µs


In [11]:
def identity_tokenizer(text):
    return text

time: 13 ms


In [12]:
#Word should occur in atleast 5% and at the most 90% of the docs
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False,min_df=0.05,max_df=0.90)
tfidf_matrix=tfidf.fit_transform(data_lemmatized)

time: 101 ms


In [13]:
tfidf_feature_names = tfidf.get_feature_names()

time: 996 µs


In [14]:
from sklearn.decomposition import NMF

time: 5.99 ms


In [15]:
no_topics = 10
# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_matrix)

time: 424 ms


In [16]:
nmf

NMF(alpha=0.1, beta_loss='frobenius', init='nndsvd', l1_ratio=0.5,
  max_iter=200, n_components=10, random_state=1, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

time: 3.99 ms


In [17]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

time: 5.98 ms


In [18]:
n_top_words=10
print("\nTopics in NMF model")
#tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model
Topic #0: go see get time crowd enjoy visit exhibit fun lot
Topic #1: great family time kid food fun staff day exhibit little
Topic #2: love kid come back family dolphin go first aquarium fish
Topic #3: amazing staff animal back exhibit much definitely worth dolphin come
Topic #4: place visit nice family beautiful wonderful kid must take fish
Topic #5: good aquarium food price kid staff really many exhibit worth
Topic #6: show dolphin sea lion see well also must get enjoy
Topic #7: shark whale tank fish see large exhibit worth sea well
Topic #8: awesome time staff definitely family little dolphin exhibit price kid
Topic #9: experience wonderful beautiful make staff visit sea come first family

time: 26.9 ms
